<a href="https://colab.research.google.com/github/simonsanvil/ECG-classification-MLH/blob/master/FeatureExtrtion_MLH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [3]:
%cd ./drive/MyDrive/MLHC/ecg-features

/content/drive/MyDrive/MLHC/ecg-features


In [ ]:
#!git clone 'https://github.com/Seb-Good/ecg-features.git'

In [4]:
!pip install biosppy

     |████████████████████████████████| 85 kB 2.5 MB/s 
  Created wheel for biosppy: filename=biosppy-0.7.3-py2.py3-none-any.whl size=95430 sha256=f0ea17de6db9af79fff681ad35f2a55dfae87ed70d4c94f5a93de7c6f825a944
  Stored in directory: /root/.cache/pip/wheels/2f/4f/8f/28b2adc462d7e37245507324f4817ce1c64ef2464f099f4f0b
Successfully built biosppy


In [5]:
# Import 3rd party libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import shutil
from sklearn.model_selection import train_test_split

# Import local Libraries
sys.path.insert(0, os.path.dirname(os.getcwd()))
from features.feature_extractor import Features
from utils.plotting.waveforms import plot_waveforms

# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
#to move all .mat files to a folder, only run it for the first time
sourcepath=os.path.join('./data','trainingData')
sourcefiles = os.listdir(sourcepath)
waveform_path = os.path.join('./data', 'waveforms')
for file in sourcefiles:
    if file.endswith('.mat'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(waveform_path,file))

In [6]:
# Sampling frequency (Hz)
fs = 300  

# Data paths
label_path = os.path.join('./data', 'labels')
waveform_path = os.path.join('./data', 'waveforms')
feature_path = os.path.join('./data', 'features')

In [7]:
# Read labels CSV, convert them to numeric
labels = pd.read_csv(os.path.join(label_path, 'REFERENCE.csv'), names=['file_name', 'label'])
## encode label
all_label = []
for i in labels['label']:
    if i == 'N':
        all_label.append(str(0))
    elif i == 'A':
        all_label.append(str(1))
    elif i == 'O':
        all_label.append(str(2))
    elif i == '~':
        all_label.append(str(3))
labels.label = all_label
# View DataFrame
labels.head(10)

,file_name,label
0,A00001,0
1,A00002,0
2,A00003,0
3,A00004,1
4,A00005,1
5,A00006,0
6,A00007,0
7,A00008,2
8,A00009,1
9,A00010,0


In [8]:
# Launch interactive plotting widget
plot_waveforms(labels=labels, waveform_path=waveform_path, fs=fs)

interactive(children=(IntSlider(value=4263, description='index', max=8527), Output()), _dom_classes=('widget-i…

In [ ]:
# Instantiate
#ecg_features = Features(file_path=waveform_path, fs=fs, feature_groups=['full_waveform_features'])

# Calculate ECG features
#ecg_features.extract_features(
#    filter_bandwidth=[3, 45], n_signals=None, show=True, 
#    labels=labels, normalize=True, polarity_check=True,
#    template_before=0.25, template_after=0.4
#)

Streaming output truncated to the last 5000 lines.
Finished extracting features from A05058.mat | Extraction time: 0.005 minutes
Finished extracting features from A05059.mat | Extraction time: 0.005 minutes
Finished extracting features from A05060.mat | Extraction time: 0.005 minutes
Finished extracting features from A05061.mat | Extraction time: 0.008 minutes
Finished extracting features from A05062.mat | Extraction time: 0.007 minutes
Finished extracting features from A05063.mat | Extraction time: 0.004 minutes
Finished extracting features from A05064.mat | Extraction time: 0.005 minutes
Finished extracting features from A05065.mat | Extraction time: 0.006 minutes
Finished extracting features from A05066.mat | Extraction time: 0.008 minutes
Finished extracting features from A05067.mat | Extraction time: 0.005 minutes
Finished extracting features from A05068.mat | Extraction time: 0.004 minutes
Finished extracting features from A05069.mat | Extraction time: 0.005 minutes
Finished extr

In [ ]:
# Get features DataFrame
#features = ecg_features.get_features()

# View DataFrame
#features.head(10)

,file_name,label,full_waveform_duration,full_waveform_kurtosis,full_waveform_max,full_waveform_mean,full_waveform_median,full_waveform_min,full_waveform_skew,full_waveform_std,swt_a_1_energy_entropy,swt_a_1_high_power_ratio,swt_a_1_higuchi_fractal,swt_a_1_low_power_ratio,swt_a_1_med_power_ratio,swt_a_2_energy_entropy,swt_a_2_high_power_ratio,swt_a_2_higuchi_fractal,swt_a_2_low_power_ratio,swt_a_2_med_power_ratio,swt_a_3_energy_entropy,swt_a_3_high_power_ratio,swt_a_3_higuchi_fractal,swt_a_3_low_power_ratio,swt_a_3_med_power_ratio,swt_a_4_energy_entropy,swt_a_4_high_power_ratio,swt_a_4_higuchi_fractal,swt_a_4_low_power_ratio,swt_a_4_med_power_ratio,swt_d_1_energy_entropy,swt_d_1_high_power_ratio,swt_d_1_higuchi_fractal,swt_d_1_low_power_ratio,swt_d_1_med_power_ratio,swt_d_2_energy_entropy,swt_d_2_high_power_ratio,swt_d_2_higuchi_fractal,swt_d_2_low_power_ratio,swt_d_2_med_power_ratio,swt_d_3_energy_entropy,swt_d_3_high_power_ratio,swt_d_3_higuchi_fractal,swt_d_3_low_power_ratio,swt_d_3_med_power_ratio,swt_d_4_energy_entropy,swt_d_4_high_power_ratio,swt_d_4_higuchi_fractal,swt_d_4_low_power_ratio,swt_d_4_med_power_ratio
0,A00001,0,29.996667,14.631998,1.121756,0.001065,-0.006663,-0.405281,3.195531,0.179318,-16622.628118,1.128280,1.040502,0.234664,0.811725,-19689.280459,1.169583,1.080389,0.267620,0.247508,-22365.502217,0.302583,1.099275,0.268713,0.193264,-25065.969065,0.244791,1.101142,0.269077,0.191679,-21176.602201,1.391602,1.115929,0.521239,0.187217,-33640.591867,0.301326,1.272694,0.706138,0.094746,-47111.349430,0.140239,1.557447,0.754295,0.081824,-65378.118352,0.087640,1.722197,0.766234,0.090588
1,A00002,0,29.996667,31.101018,2.627004,0.001680,-0.005670,-0.954112,3.846891,0.217056,-17515.672777,1.028629,1.039355,0.228670,0.675113,-19556.292347,1.110719,1.103425,0.233224,0.143971,-22054.070992,0.245347,1.149023,0.233371,0.103805,-24747.280306,0.196029,1.160585,0.233559,0.103395,-22018.832647,1.373491,1.136383,0.407693,0.147860,-32658.667549,0.234504,1.324742,0.593482,0.087809,-46065.956719,0.112684,1.644520,0.642252,0.116674,-64265.127385,0.099459,1.802574,0.653203,0.158843
2,A00003,0,59.996667,13.045449,1.404153,0.000763,-0.015898,-0.632330,2.797258,0.201006,-26638.875785,1.080695,1.037991,0.222158,0.718759,-31420.169390,1.138247,1.092252,0.251069,0.172418,-36491.704813,0.303651,1.123247,0.251810,0.126792,-41836.760207,0.249265,1.127966,0.252234,0.125710,-35862.910152,1.383370,1.128560,0.479706,0.153769,-58329.052960,0.303432,1.295171,0.667884,0.088451,-85876.368610,0.146982,1.589489,0.717435,0.090127,-122690.207723,0.093896,1.751607,0.729590,0.105205
3,A00004,1,29.996667,23.507523,1.097179,0.002550,-0.006243,-0.295871,4.208181,0.148257,-18711.372531,1.088687,1.043470,0.241216,0.789751,-21536.939962,1.142020,1.083964,0.232422,0.225770,-24159.785056,0.313575,1.115416,0.233139,0.164630,-26860.390936,0.262473,1.125487,0.233772,0.162606,-25761.919907,1.403319,1.122565,0.502945,0.172237,-39624.542164,0.313335,1.309092,0.721162,0.081516,-52921.488662,0.160702,1.595750,0.778542,0.096383,-70980.710219,0.107169,1.738679,0.792823,0.126350
4,A00005,1,59.996667,12.890250,1.196451,0.000772,-0.008146,-0.766196,2.933488,0.192305,-31980.351872,1.109665,1.042481,0.233371,0.692102,-36705.199517,1.164647,1.100447,0.270721,0.175106,-41972.582987,0.307184,1.143862,0.272828,0.118387,-47413.113336,0.254397,1.161286,0.273704,0.116793,-40633.035994,1.411692,1.134060,0.499246,0.155978,-65665.560318,0.305095,1.334632,0.673172,0.082437,-90667.220615,0.152443,1.635041,0.719155,0.113764,-126213.720375,0.103769,1.780584,0.730294,0.148307
5,A00006,0,29.996667,20.278696,1.072912,0.000341,0.001891,-0.489380,3.266021,0.149187,-21048.578121,1.103964,1.046558,0.287895,0.740101,-24048.404160,1.156558,1.096528,0.260420,0.172785,-26640.379129,0.318587,1.118466,0.263121,0.126847,-29379.554642,0.266372,1.122107,0.263411,0.125651,-27003.941936,1.373615,1.126694,0.664370,0.156537,-40995.859973,0.317825,1.278785,0.873396,0.093353,-54676.387529,0.162565,1.525826,0.922243,

In [ ]:
# Save features DataFrame to CSV
features.to_csv(os.path.join(feature_path, 'newFeatures.csv'), index=False)

In [9]:
# split train val test
features=pd.read_csv(os.path.join(feature_path, 'newFeatures.csv'))
Y=features.label
X=features.drop(['label'],axis=1)
X_train, X_test = train_test_split(features, test_size=0.2, random_state=0)
X_val, X_test = train_test_split(X_test, test_size=0.5, random_state=0)

###Using H2O to try different models

In [15]:
!pip install requests
!pip install tabulate
!pip install future

In [16]:
!pip uninstall h2o
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 175.8 MB 24 kB/s 
  Created wheel for h2o: filename=h2o-3.34.0.3-py2.py3-none-any.whl size=175832480 sha256=4f8c1ed55f707f37ee72e9a71fdf01a8bbd197be1e1d38cecb69c274121a6bb8
  Stored in directory: /root/.cache/pip/wheels/2a/52/cf/01f6938288f0731963463ff72510db67f9f3076f801038b2f7
Successfully built h2o


In [ ]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
train = h2o.H2OFrame(X_train) #X_train[['hosp_exp_flg','mort_day_censored']]
test = h2o.H2OFrame(X_test)
val= h2o.H2OFrame(X_val)
# Identify predictors and response
x = ['full_waveform_kurtosis','full_waveform_max','full_waveform_mean','full_waveform_median','full_waveform_min','full_waveform_skew','full_waveform_std','swt_a_2_energy_entropy','swt_a_2_high_power_ratio','swt_a_2_higuchi_fractal','swt_a_2_low_power_ratio','swt_a_2_med_power_ratio']
y = "label"
#x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=x, y=y, training_frame=train)
#aml.predict(test)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,17 mins 04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_unknownUser_m2h78a
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.042 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_AllModels_2_AutoML_2_20211130_231004,0.511134,0.804694,0.526452,0.277151
StackedEnsemble_BestOfFamily_3_AutoML_2_20211130_231004,0.517155,0.804815,0.526973,0.277701
StackedEnsemble_AllModels_1_AutoML_2_20211130_231004,0.517343,0.805362,0.526939,0.277665
StackedEnsemble_AllModels_4_AutoML_2_20211130_231004,0.517987,0.823271,0.530585,0.281521
StackedEnsemble_BestOfFamily_2_AutoML_2_20211130_231004,0.519657,0.805263,0.527527,0.278285
StackedEnsemble_AllModels_5_AutoML_2_20211130_231004,0.522643,0.800284,0.527733,0.278502
StackedEnsemble_BestOfFamily_6_AutoML_2_20211130_231004,0.523545,0.804111,0.528397,0.279204
StackedEnsemble_BestOfFamily_5_AutoML_2_20211130_231004,0.524008,0.824207,0.530778,0.281726
StackedEnsemble_BestOfFamily_1_AutoML_2_20211130_231004,0.526672,0.809889,0.529997,0.280897
XGBoost_1_AutoML_2_20211130_231004,0.527354,0.86036,0.536372,0.287695


In [ ]:
aml.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,A,N,O,~
N,0.0358917,0.765898,0.1976,0.000610756
N,0.0107528,0.826287,0.162379,0.000581275
N,0.0523392,0.690129,0.25561,0.00192202
N,0.0860176,0.675192,0.238034,0.00075659
N,0.0254626,0.770135,0.204097,0.000304666
N,0.0279545,0.740781,0.225558,0.00570647
O,0.0720692,0.413075,0.509984,0.00487176
N,0.0371478,0.795861,0.166169,0.000821717
~,0.085586,0.224102,0.167017,0.523296
N,0.0358505,0.748401,0.213877,0.00187096


In [ ]:
print(X_test2.label)

4773    N
3424    N
3338    O
4089    N
2990    N
       ..
6782    N
8351    N
6393    N
4724    N
4139    N
Name: label, Length: 853, dtype: object


In [22]:
from h2o.estimators import H2OXGBoostEstimator
import h2o
h2o.init()
# Import the dataset
x = h2o.import_file(os.path.join(feature_path, 'newFeatures.csv'))

# Set the predictor names and the response column name

predictors = ['full_waveform_kurtosis','full_waveform_max','full_waveform_median','full_waveform_min','full_waveform_std','swt_a_2_energy_entropy','swt_a_2_higuchi_fractal','swt_a_2_low_power_ratio','swt_a_2_med_power_ratio']
response = "label"
#predictors.remove(response)

# Convert the response column to a factor
x['label'] = x['label'].asfactor()

# Train a GBM model setting nfolds to 6
xgb = H2OXGBoostEstimator(nfolds = 6, seed = 1, ntrees=2000)
xgb.train(x=predictors, y=response, training_frame=x)

# metrics of cross-validated holdout predictions
xgb.cross_validation_metrics_summary


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,19 mins 28 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 25 days
H2O_cluster_name:,H2O_from_python_unknownUser_eoqzfs
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.146 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1638458728381_4


Model Summary: 


,,number_of_trees
0,,2000.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 1.5983788497664916e-05
RMSE: 0.003997973048641639
LogLoss: 0.0031051285339371987
Mean Per-Class Error: 0.0
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2021-12-02 15:53:28,8 min 28.032 sec,0.0,0.750000,1.386294,0.712008,NaN,NaN
1,,2021-12-02 15:53:28,8 min 28.113 sec,1.0,0.690347,1.180670,0.319653,NaN,NaN
2,,2021-12-02 15:53:28,8 min 28.172 sec,2.0,0.645700,1.055402,0.311327,NaN,NaN
3,,2021-12-02 15:53:29,8 min 28.230 sec,3.0,0.611076,0.968553,0.303823,NaN,NaN
4,,2021-12-02 15:53:29,8 min 28.301 sec,4.0,0.583177,0.902457,0.300070,NaN,NaN
5,,2021-12-02 15:53:29,8 min 28.364 sec,5.0,0.560625,0.850483,0.293504,NaN,NaN
6,,2021-12-02 15:53:29,8 min 28.427 sec,6.0,0.543054,0.810593,0.289634,NaN,NaN
7,,2021-12-02 15:53:29,8 min 28.490 sec,7.0,0.529583,0.779976,0.284592,NaN,NaN
8,,2021-12-02 15:53:29,8 min 28.559 sec,8.0,0.518262,0.754058,0.284006,NaN,NaN
9,,2021-12-02 15:53:29,8 min 28.628 sec,9.0,0.507967,0.730304,0.279550,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,swt_a_2_energy_entropy,4371.618652,1.000000,0.146318
1,full_waveform_std,3638.646973,0.832334,0.121785
2,swt_a_2_higuchi_fractal,3372.185303,0.771381,0.112867
3,full_waveform_min,3312.384277,0.757702,0.110865
4,full_waveform_kurtosis,3242.537354,0.741725,0.108528
5,swt_a_2_low_power_ratio,3135.282227,0.717190,0.104938
6,full_waveform_median,3028.832764,0.692840,0.101375
7,swt_a_2_med_power_ratio,2969.036133,0.679162,0.099373
8,full_waveform_max,2807.027588,0.642103,0.093951


<bound method ModelBase.cross_validation_metrics_summary of >

###Xgboost package

In [11]:
#https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [24]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))



def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")



In [12]:
y=X_train.label
X = X_train[['full_waveform_kurtosis','full_waveform_max','full_waveform_mean','full_waveform_median','full_waveform_min','full_waveform_skew','full_waveform_std','swt_a_2_energy_entropy','swt_a_2_high_power_ratio','swt_a_2_higuchi_fractal','swt_a_2_low_power_ratio','swt_a_2_med_power_ratio']]


xgb_model = xgb.XGBClassifier()

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 1),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(20, 60), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=50, cv=10, verbose=1, n_jobs=4, return_train_score=True)

search.fit(X, y)

report_best_scores(search.cv_results_, 1)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


KeyboardInterrupt: ignored

###Using sktime and TFresh

In [ ]:
!pip install sktime
!pip install --upgrade tsfresh
#https://www.sktime.org/en/latest/examples/feature_extraction_with_tsfresh.html

  Using cached statsmodels-0.13.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.8 MB)
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.8.1 requires statsmodels<=0.12.1, but you have statsmodels 0.13.1 which is incompatible.


In [ ]:
def featureExtraction(X):
    t = TSFreshFeatureExtractor(default_fc_parameters="efficient", show_warnings=False)
    X = t.fit_transform(X)
    return X


In [ ]:
import pickle
import os
import pandas as pd
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor
#with open(os.path.join('./data/files','challenge2017.pkl'), 'rb') as fin:
data = pd.read_pickle(os.path.join('./data/files','challenge2017.pkl'))
X = data['data']
Y=data['label']
F=featureExtraction(X)

ValueError: ignored